In [1]:
# Loading libraries

import nltk
import pandas as pd
import numpy as np
import csv

In [2]:
# Load data
# Rename columns

df = pd.read_csv(r"D:\SLU\AI MSc\Fall 22\NLP\Project 2\train.tsv", sep="\t", header=None, quoting = csv.QUOTE_NONE)
df.rename(columns={0:"Word", 1:"Label"}, inplace=True)

In [3]:
# Inspecting Null values

df.isnull().sum()

Word     27
Label     0
dtype: int64

In [4]:
# Removing null records

df.dropna(inplace=True)

In [5]:
# test, train split

from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=42)
print(len(train))
print(len(test))

7999978
1999995


In [6]:
# Getting words with labels in the shape of List of List of Tuples

llt = [[(train.loc[i, 'Word'], train.loc[i, 'Label'])] for i in train.index]


In [7]:
# Getting words with labels in the shape of List of Tuples

lt = [(train.loc[i, 'Word'], train.loc[i, 'Label']) for i in train.index]

In [8]:
# find the frequency of words

fd = nltk.FreqDist(tag for [(word,tag)] in llt)
fd

FreqDist({'N': 6867774, 'S': 778541, 'U': 261623, 'H': 64255, 'T': 27785})

In [9]:
# conditional freqency

cfd = nltk.probability.ConditionalFreqDist(lt)
print(cfd['cearta']['U'])
print(cfd['cearta']['S'])
print(cfd['cearta']['N'])
print(cfd['cearta']['H'])
print(cfd['cearta']['T'])

102
577
1104
0
0


In [10]:
# list of words in train and test data

train_words_list = train['Word'].values.tolist()
test_words_list = test['Word'].values.tolist()

In [11]:
# keys of words in train and test data

train_justwords = nltk.FreqDist(train_words_list).keys()
test_justwords = nltk.FreqDist(test_words_list).keys()

In [12]:
# fit on train

unigram_tagger = nltk.UnigramTagger(llt)

In [13]:
# Prediction process

test['Predict'] = None      # adding one new column to test data: 'Predict'
tags_list = ['N', 'S', 'T', 'U', 'H']
for i in test.index:
    token = test.loc[i, 'Word']     # selecting words in test data one by one
    prediction = unigram_tagger.tag([token])[0][1]      # tagging with nltk
    # from my initial runs, I noticed that some of the tokens are not tagged with the algorithm.
    # So here I check and if it is not predicted, I tagged them manually based on CFD.
    if prediction not in tags_list:
        tags_dist = {i:cfd[token][i] for i in tags_list}
        prediction = max(tags_dist, key=tags_dist.get)
    test.loc[i, 'Predict'] = prediction     # write predicted tag in the 'Predict' column

In [14]:
# Check accuracy

len(test[test['Label'] == test['Predict']]) / len(test)

0.9079132697831744